In [4]:
# imports
import hashlib
import shutil
from pathlib import Path

from tqdm.auto import tqdm

In [5]:
# set root_dir and meta_yml paths
root_dir_path = Path('/Users/dlisla/shared_with_docker/agrtfhs/3.toQC')
meta_yml_path = Path('/Users/dlisla/shared_with_docker/agrtfhs/meta.yml')
root_dir_string = str(root_dir_path)
root_dir_path

PosixPath('/Users/dlisla/shared_with_docker/agrtfhs/3.toQC')

In [6]:
# functions
def batch_process(root_dir):
    
    root_dir_path = Path(root_dir)
    dir_path_list = sorted([x for x in root_dir_path.iterdir() if x.is_dir()])
    number_of_dir = len(dir_path_list)
    dir_path_tqdm_desc = f'Batch processing {number_of_dir} directories'
    
    for dir_path in tqdm(dir_path_list, desc=dir_path_tqdm_desc):
        
        create_sip_tif(dir_path)      

def create_sip_tif(directory, overwrite=True):
    '''Creates SIP from a directory containing *.tif files
    for HathiTrust ingest'''
    
    
    input_dir_path = Path(directory)
    tif_path_list = get_file_list(input_dir_path, 'tif')
    number_of_tif_input = len(tif_path_list)
    #tqdm.write(str(tif_path_list[0]))
    tqdm.write(f'\nSIPing ({number_of_tif_input} images): {directory}')
    
    # copy *.tif into a new SIP/dir_path.name directory 1 level up
    # from the input directory so we can run batches
    tqdm.write(f'\n\t\tCopying/renaming . . . ')
    dir_path = input_dir_path.parents[1].joinpath('SIP', input_dir_path.name)
    if overwrite:
        exist_ok = True
    else:
        exist_ok = False
    dir_path.mkdir(parents=True, exist_ok=exist_ok)
    
    for i, tif_path in tqdm(enumerate(tif_path_list, start=1), leave=False, total=number_of_tif_input):
        new_tif_name = str(i).zfill(8) + tif_path.suffix
        new_tif_path = dir_path.joinpath(new_tif_name)
        
        if overwrite:
            shutil.copy(tif_path, new_tif_path)
        else:  # don't overwrite any existing files
            shutil.copyfile(tif_path, new_tif_path)
        
    tif_path_list = get_file_list(dir_path, 'tif')
    #tqdm.write(str(tif_path_list[0])
    number_of_tif = len(tif_path_list)
    if number_of_tif != number_of_tif_input:
        raise ValueError
    
    # jhove validation
    tqdm.write(f'\t\tjHoveing . . . ')
    jhove_error_dict = jhove_batch_tif(tif_path_list)

    if jhove_error_dict:
        tqdm.write(f'errors: {jhove_error_dict}\n')
        return jhove_error_dict

    # OCR
    tqdm.write(f'\t\tOCRing . . .')
    ocr_batch_tif(tif_path_list)
    
    # copy meta.yml
    sip_meta_yml_path = dir_path.joinpath(meta_yml_path.name)
    shutil.copy(meta_yml_path, sip_meta_yml_path)
    
    # create checksum.md5
    tqdm.write(f'\t\tHashing . . .')
    get_checksum_md5(dir_path)
    
    # zip directory
    tqdm.write(f'\t\tZipping {dir_path.name} . . .')
    !zip -r {str(dir_path)}.zip {str(dir_path)} 1>/dev/null 2>&1
    
    if Path(f'{str(dir_path)}.zip').is_file():
        tqdm.write(f'\t\t. . .  {dir_path.name} Zipped')
              
def delete_dot_files(directory):
    
    dir_path = Path(directory)
    
    dot_file_path_list = list(dir_path.glob('**/.*'))

    if dot_file_path_list:  # then delete!
        number_of_dot_files = len(dot_file_path_list)
        for dot_file_path in dot_file_path_list:
            dot_file_path.unlink()
        dot_file_path_list = list(dir_path.glob('**/.*'))
        number_of_dot_files_deleted = number_of_dot_files = len(dot_file_path_list)
        tqdm.write(f'Deleted: {number_of_dot_files_deleted}')
    
def get_file_list(dir_path, file_type, recursive=False, del_dot_files=True):
    
    # format & remediate extension
    glob = '*' + get_formatted_extension('tif', remediate=True)
    
    if recursive:  # search into directories
        glob = '**/' + glob
        
    #tqdm.write(glob)
    
    if del_dot_files:  # delete them dot files!
        file_path_list = list(dir_path.glob(glob))
        delete_dot_files(dir_path)
        file_path_list = sorted(dir_path.glob(glob))
    
    # return sorted file path list
    return file_path_list
    
def get_formatted_extension(from_extension, remediate=False):
    '''
    -- Purpose --
    Returns an extension that:
    1. has a period in the front
    2. Optional: is lower-case
    3. Optional: return jpeg as jpg and tiff as tif

    -- Arguments --
    from_extension: type=string; file extension with or without a '.'

    -- Returns --
    formatted_extension: type=string; formatted extension
    '''
    # make sure there's a period at the front of the extension
    if from_extension.startswith('.'):  # do nothing
        formatted_extension = from_extension
    else:  # add a period
        formatted_extension = f'.{from_extension}'

    # make it lower-case
    if remediate:
        formatted_extension = formatted_extension.lower()
        # hard-coded alterations for jpeg and tiff
        if formatted_extension == '.jpeg':
            formatted_extension = '.jpg'
        elif formatted_extension == '.tiff':
            formatted_extension = '.tif'

    return formatted_extension

def get_checksum_md5(directory, delete_previous=True):
    
    dir_path = Path(directory)
    checksum_md5_path = dir_path.joinpath('checksum.md5')
    
    if delete_previous and checksum_md5_path.is_file():  # delete it
        checksum_md5_path.unlink()
    
    file_path_list = [x for x in dir_path.iterdir() if x.is_file()]
    number_of_file = len(file_path_list)
    
    for file_path in tqdm(file_path_list, leave=False):
        # Python method, but md5 worked for the pilot
        # hash_md5 = get_hash_md5(file_path)
        
        # terminal md5 used for pilot
        !md5 -r {str(file_path)} >> {str(checksum_md5_path)}
        
def get_hash_md5(file_path):
    hash_md5 = hashlib.md5()
    
    with open(file_path, 'rb') as file:
        for chunk in iter(lambda: file.read(4096), b""):
            hash_md5.update(chunk)
            
    return hash_md5.hexdigest() 

def jhove_batch_tif(tif_path_list):
    
    # jhove location and command
    jhove = f'~/jhove/jhove -m TIFF-hul'

    # instantiate error dictionary
    error_dict = {}
    number_of_tif = len(tif_path_list)
    
    for tif_path in tqdm(tif_path_list, leave=False):
        status = !{jhove} "{str(tif_path)}" | grep -e 'Status:'
        status = status[0]
        if status == '  Status: Well-Formed and valid':
            continue
        else:
            error_dict.update( {tif_path.name : status} )

    return error_dict

def ocr_batch_tif(tif_path_list, xml=True):
    
    # need to:
    ##### OCR tif into *.txt & *.hocr
    ##### Rename *.hocr to *.xml
    ##### Clean up *.txt
    
    # progress bar description
    number_of_tif = len(tif_path_list)
    # ocr_tqdm_desc = f'OCRing {number_of_tif} *.tif\n'
    
    for tif_path in tqdm(tif_path_list, leave=False):
        
        # set names & paths
        hocr_name = tif_path.stem + '.hocr'
        hocr_path = tif_path.parent.joinpath(hocr_name)
        txt_name = tif_path.stem + '.txt'
        txt_path = tif_path.parent.joinpath(txt_name)
        xml_name = tif_path.stem + '.xml'
        xml_path = tif_path.parent.joinpath(xml_name)
        
        if not xml_path.is_file():  # no need to re-OCR something we've OCRed!
            
            if not txt_path.is_file():  # OCR
                !tesseract {str(tif_path)} {tif_path.parent.joinpath(tif_path.stem)} 1>/dev/null 2>&1
                
            # convert form feed to line feed
            text_clean = []
            
            with open(txt_path) as txt_read:
                text_to_clean = txt_read.readlines()
                
            for line in text_to_clean:
                line_clean = []
                
                for char in line:
#                     if char not in string.printable:
#                         return char
                    if char == '\x0c':  # form feed
                        char = '\n'
                    line_clean.append(char)
                    
                text_clean.append(''.join(line_clean))
                
            with open(txt_path, 'w') as txt_write:
                for line in text_clean:
                    txt_write.write(line)
                
            if not hocr_path.is_file():  # OCR and rename to .xml
                !tesseract {str(tif_path)} {tif_path.parent.joinpath(tif_path.stem)} hocr 1>/dev/null 2>&1
                
                if xml:
                    hocr_path.rename(xml_path)

In [8]:
batch_process(root_dir_path)


SIPing (12 images): /Users/dlisla/shared_with_docker/agrtfhs/3.toQC/0012_004267_000001

		Copying/renaming . . . 


		jHoveing . . . 


		OCRing . . .


		Hashing . . .


		Zipping 0012_004267_000001 . . .
		. . .  0012_004267_000001 Zipped

SIPing (12 images): /Users/dlisla/shared_with_docker/agrtfhs/3.toQC/0012_004267_000002

		Copying/renaming . . . 


...

		Zipping 0012_004267_000134 . . .
		. . .  0012_004267_000134 Zipped

SIPing (32 images): /Users/dlisla/shared_with_docker/agrtfhs/3.toQC/0012_004267_000135

		Copying/renaming . . . 


		jHoveing . . . 


		OCRing . . .


		Hashing . . .


		Zipping 0012_004267_000135 . . .
		. . .  0012_004267_000135 Zipped

